# Projects settings

La prima cosa da fare per un nuovo progetto è quella di creare un ambiente virtuale. Questo ci consente di lavorare con versioni specifiche di librerie e dell'interprete che più riteniamo ottimali per la buona riuscita del progetto. Farò tutti i passaggi necessari tramite il terminale.

<b>Creazione ambiente virtuale</b><br>
python -m venv venv

<b>Attivazione dell'ambiente virtuale</b><br>
./Scripts/Activate.ps1

<b>Installazione pacchetti e kernel</b><br>
pip install pandas<br>
pip install numpy<br>
pip install sidetables<br>
pip install Jinja2<br>
pip install ipykernel<br>
pip install plotly==5.11.0<br>
pip install nbformat<br>
pip install statsmodels

<b>Impostazione del kernel per il notebook</b><br>
ipython kernel install --user --name=venv

<b>Creazione di un file .txt contenente i pacchetti e le versioni</b><br>
pip freeze > requirements.txt

# Discovery
<p>L'obiettivo di questa analisi è quello di creare un catalogo per un e-commerce di vini<p>
Il catalogo sarà costituito da 2 categorie:
<ul>
<li>Catalogo nazioni</li>
<li>Catalogo varietà</li>
</ul><br>
Ogni categoria avrà 3 vini per componente distribuiti per fascia di prezzo

# Data Selection

In [ ]:
# Standard libraries 
import numpy as np
import pandas as pd
import math
import scipy.stats as st

# Data cleaning libraries
import sidetable
from re import search
from datetime import date

# Visualization libraries
import plotly 
import plotly.express as px



In [ ]:
# Importing data
df_raw = pd.read_csv('winemag-data-130k-v2.csv')

# Creating a copy of the raw data to mantain the original version just in case
df = df_raw.copy()
df.head()

# Data Cleaning

In [ ]:
# Rows and columns shape
df.shape

Shape ci sta un idea della struttura del dataframe, in questo caso il nostro  set di dati è composto da 129.997 righe e 14 colonne

In [ ]:
# Basic statistic
df.describe()

Describe ci sta una prima infarinatura di statistiche di base sulle colonne numeriche del dataset

In [ ]:
# Columns types and non-null count
df.info()

Info invece ci da informazioni riguardo il tipo di variabili che abitano il nostro set di dati e un count delle entità

E' buona norma controllare la presenta di duplicati all'interno del set. In questo caso ho creato una funzione personalizzata che mi stampa il numero di righe prima e dopo la rimozione dei duplicati, in caso ce ne siano la funzione li rimuove e me lo comunica, altrimenti mi dirà che non sono stati trovati duplicati

In [ ]:
# Cheking for duplicates
def duplicates_check(dataframe):
    # Variable that store the number of rows before removing duplicates
    before = dataframe.shape[0]
    print(f'Number of rows before dropping duplicates: {before:>6}')

    # removing duplicates if there any
    dataframe.drop_duplicates(keep='first', inplace=True)

    # variables that stores the number of rows after removing duplicates
    after = dataframe.shape[0]
    print(f'Number of rows after dropping duplicates: {after :>7}')

    # printing the result
    if before == after:
        print('No duplicates were found')
    else:
        print(f'{before - after} duplicates were found and removed')
    return 

duplicates_check(df)

Le seguenti funzioni mi serviranno per estrarre l'anno di produzione del vino dall'etichetta 'title'. La prima funzione prende come input una stringa (l'etichetta della colonna 'title' appunto) e ritorna un valore booleano in caso in cui sia presente qualsiasi numero all'interno dell'etichetta. Ho applicato questa funzione tramite la funzione di pandas apply() alla colonna delle cantine, creando una condizione che ho poi usato per filtrare il set per ottenere una lista di tutte le cantine che hanno all'interno del loro nome un numero. Questo è stato necessario in quanto dopo una prima applicazione della seconda funzione, mi sono accorto che all'interno di alcune etichette era presente sia l'anno di produzione del vino che l'anno presente nel nome della cantina. 

In [ ]:
def winery_solver(string):
    return any(x.isdigit() for x in string.split())

cond = df['winery'].apply(winery_solver)

df.loc[cond, 'winery'].unique()

Una volta ottenuta la lista delle cantine problematiche ho ciclato al suo interno. Mi interessa controllare se il nome di una di queste cantine sia presente all'interno dell'etichetta che sto processando, e in caso affermativo, sostituire alla cantina la stringa vuota ''. In questo modo elimino il problema del doppio anno e posso poi procedere all'estrazione. Splitto la stringa che compone l'etichetta usando lo spazio come separatore in singole stringhe che mi costruiscono una lista. Ciclando questa lista, se l'elemento <i>s</i> è una cifra e la sua lunghezza è uguale a 4, lo trasformo in un intero. Poi controllo che il suo valore sia maggiore di 1796 (bottiglia di vino più antica attualmente in commercio) e che sia minore dell'anno attuale (impossile per quanto ne sappiamo al momento produrre un vino nel futuro). Se l'elemento <i>s</i> soddisfa tutte queste condizioni la funzione ritorna l'elemento <i>s</i>. Ho poi applicato la funzione con apply() alla colonna 'title' creando così la colonna 'year' con l'anno di produzione del vino.

In [ ]:
# extracting the year of the wine from the title column

# creating the function
def num_extract(string: str):
    wineries = [
        'Antichi Vinai 1877',
        'Caccia al Piano 1868',
        'Donna Olimpia 1898',
        'Don Cristobal 1492',
        'Bodegas 1898',
        'Foxen 7200',
        'Hazlitt 1852 Vineyards',
        'Guidi 1929',
        '1912 Winemakers',
        'Baglio Curatolo Arini 1875',
        '1848 Winery',
        'Dogliotti 1870',
        'Estate 1856',
        'Tenuta Sarno 1860',
        '1850',
        '1000 Stories',
        'Prospect 1870',
        '1070 Green',
        'Bagrationi 1882',
        '1752 Signature Wines',
        '1789 Wines',
        'Fortune 1621',
        'Poderi dal Nespoli 1929',
        'Ippolito 1845',
        'Cristobal 1492',
        'Ikal 1150',
        '3000 BC',
        'Cavas Hill NV 1887',
        'Codorníu NV Reserva Cuvée Barcelona 1872',
        'Wiese & Krohn 1964',
        'Taylor Fladgate 1967',
        'Osborne NV Pedro Ximenez 1827',
        'Cavas Hill NV 1887',
        'Codorníu NV Reserva Cuvée Barcelona 1872',
        "Blandy's 1969",
        'Mas de Monistrol NV MM Masia 1882',
        'Kopke 1935 Colheita White  (Port)',
        'Zonin NV Cuvèe 1821',
        'Zonin NV Cuvèe 1821',
        'Kopke 1965',
        'Taylor Fladgate 1968',
        'Bodegas Toro Albala, SL 1947',
        "Dow's 1963",
        'Alvear NV Solera 1927',
        'Taylor Fladgate 1964',
        "Arthur Metz NV 1904",
        'Carpenè Malvolti NV 1868',
        'González Byass NV Solera 1847',
        'Messias 1963',
        "L'Arboc NV 1919",
        'Burmester 1963',
        'Kopke 1957',
        'Kopke 1966',
        'Cálem 1961',
        'Messias 1952',
        'Zonin NV Cuvée 1821',
        'Osborne NV Premium Sweet Sherry 1827',
        'Burmester 1952',
        'Kopke 1941',
        'Taylor Fladgate 1966',
        'Adega Viuva Gomes 1934',
        'Gérard Bertrand 1945',
        'Kopke 1935'
        ]


    for t in wineries:
        if search(t, string):
            string = string.replace(t, '')   
    for s in string.split():
        if s.isdigit() and len(s) == 4:
            s = int(s)
            if s > 1796 and s <= date.today().year:
                return s

# creating the column by applying the function to the title column and changing the type from float to int
df['year'] = df['title'].apply(num_extract).astype('Int64')

Procedo poi a controllare la presenza di valori nulli

In [ ]:
# Checking for missing values in the dataset
df.stb.missing(style=True)

Le colonne designation e region_1 hanno rispettivamente il 28.79% e il 16% di dati mancanti, ma per la mia analisi sono colonne trascruabuli.<br>
I prezzi invece hanno il 7% di dati mancanti, proverò a prevedere quei prezzi.

Droppo quindi le colonne che so che non mi serviranno.

In [ ]:
# Dropping unnecessary columns
df.drop(['Unnamed: 0', 'designation', 'province', 'region_1', 'region_2', 'taster_name', 'taster_twitter_handle'], axis=1, inplace=True)

# Data Exploration

La prima variabile che mi interessa è quella dei punti, faccio un istogramma per vederne la distribuzione

In [ ]:
# Points Histogram
fig = px.histogram(
    df, 
    x=df['points'],
    marginal="violin",
    hover_data=df.columns,
    template='simple_white',
    width=900
)

# Layout
fig.update_layout(
    xaxis=dict(
        title='Points distribution'
    ),
    yaxis=dict(
        title='Count'
    )
)
    
fig.show()

In [ ]:
df['points'].describe()

media (88.44) e mediana (88) sono uguali, il che giustifica la poca presenza di outliers. La moda inoltre è anch'essa 88. La deviazione standard è di 3.09 punti rispetto alla media.<br>
<i>points</i> ≈ N(88.44, 3.09²)

In [ ]:
# Price distribution

# number of bins with Freedman-Diaconis rule
q1 = df['price'].quantile(0.25)
q3 = df['price'].quantile(0.75)
iqr = q3 - q1
bin_width = (2 * iqr) / (len(df['price']) ** (1 / 3))
bin_count = int(np.ceil((df['price'].max() - df['price'].min()) / bin_width))




# Histogram
fig = px.histogram(
    df, 
    x=df['price'],
    marginal="violin",
    hover_data=df.columns,
    template='simple_white',
    nbins=bin_count,
    width=900
)

# Layout
fig.update_layout(
    xaxis=dict(
        title='Price distribution'
    ),
    yaxis=dict(
        title='Count'
    )
)
    
fig.show()

La varibaile dei prezzi presenta un istogramma di difficile lettura a causa del grande range, anche se passando il mouse sul violino possiamo vedere inizialmente minimi, massimi e IQR

In [ ]:
df['price'].describe()

Con la funzione describe rendo più chiara ed immediata la situazione

Provo quindi a creare un nuovo istrogramma questa volta ignorando i valori di prezzo sopra l'upperfence (79 dollari) e risistemando il numero e la larghezza dei bins

In [ ]:
# number of bins with Freedman-Diaconis rule

no_outlier = df.loc[np.where(df['price'] <= 79)]

q1 = no_outlier['price'].quantile(0.25)
q3 = no_outlier['price'].quantile(0.75)
iqr = q3 - q1
bin_width = (2 * iqr) / (len(no_outlier['price']) ** (1 / 3))
bin_count = int(np.ceil((no_outlier['price'].max() - no_outlier['price'].min()) / bin_width))


# Histogram
fig = px.histogram(
    no_outlier, 
    x=no_outlier['price'],
    marginal="violin",
    hover_data=df.columns,
    template='simple_white',
    nbins=bin_count,
    width=900
)

# Layout
fig.update_layout(
    xaxis=dict(
        title='Price distribution'
    ),
    yaxis=dict(
        title='Count'
    )
)
    
fig.show()

Da questo istogramma si può notare come i prezzi a cifra tonda siano in netta maggioranza

In [ ]:
no_outlier['price'].describe()

Come si evince dai grafici predenti, 79 è l'upper fence di prezzo, considererò questa per la scelta dei miei range di prezzo

Ora mi interessa creare una nuova colonna per il dataset dove andrò ad inserire ogni vino in un range di prezzo, per poi vedere la distribuzione dei range.
Ho individuato 10 range di prezzo seguendo gli istogrammi precedenti con un range di 10 unità per ogni intervallo. 

In [ ]:
# Creating price_range column
df['price_range'] = df['price'].apply(lambda x: 'More than 79' if x > 79 else '70-79' if x > 69 else '60-69' if x > 59 else '50-59' if x > 49 else '40-49' if x > 39 else '30-39' if x > 29 else '20-29' if x > 19 else '10-19' if x > 9 else '0-10' if x > 0 else 'Uknown')

# Histogram
fig = px.histogram(
    df, 
    x=df['price_range'],
    y=df['price_range'],
    histfunc='count',
    template='simple_white',
    text_auto=True,
    width=900
)

# Layout
fig.update_layout(
    xaxis=dict(
        title='Price distribution',
        categoryorder='category ascending'
    ),
    yaxis=dict(
        title='Count'
    )
)

# Printing plot
fig.show()

In [ ]:
df['price_range'].describe()

In [ ]:
# Value counts (%)
round(df['price_range'].value_counts(True) * 100, 2)

Il 50% dei vini ha un prezzo compreso tra 10 e 29 dollari. Il 70% dei vini ha un prezzo compreso tra 10 e 49 dollari. Sorprendentemente è più probabile che un vino costi più di 79 dollari piuttosto che meno di 10

## Nazioni

Per costruire la parte di  catalogo dell'assortimento per le migliori nazioni mi interessa individuare quali sono le migliori nazioni basandomi sul punteggio medio ottenuto dai rispettivi vini

In [ ]:
df.groupby('country')['points'].mean().to_frame(name='mean').sort_values(by='mean', ascending=False)

l'Inghilterra è risultata al primo posto avendo però meno di 100 vini recensiti, inoltre ben 4 nazioni sono risultate tra i migliori 20 paesi avendo meno di 10 vini.<br>
Sorprendentemente Italia e Francia, i paesi più famosi al mondo per la produzione di vino non si trovano nemmeno nella top 5, con l'Italia addirittura fuori dalla top 10


Investighiamo ulteriormente, e oltre alle media vediamo quali sono i punteggi massimi e minimi ottenuti dalle nazioni ordinando per punteggio massimo e poi per media a parità di punteggio massimo.

In [ ]:
df.groupby('country')['points'].agg(['min', 'max', 'mean']).sort_values(by=['max', 'mean'], ascending=[False, False])

La classifica prende una piega diversa e più simile a quanto ci si aspettava, l'inghilterra scivola all'11esimo posto con Francia e Italia nella top 5.

Voglio indagare e mi interessa sapere come sia la distribuzione dei punti per ogni nazione

In [ ]:
fig = px.histogram(df, x='points', y='points', histfunc='count', facet_col='country', facet_col_wrap=5, height=900, width=900)

fig.show()

Gli istogrammi rivelano che alcuni paesi risultano tra i migliori avendo addirittura un solo vino, controllo quindi il conteggio di vini per paese, per poi ridurre il campione delle nazione che userò

In [ ]:
df.groupby('country')['title'].nunique().sort_values(ascending=False)

In [ ]:
round(df.groupby('country')['title'].nunique() / df['title'].nunique() * 100, 2).sort_values(ascending=False)

L'US ha 50.229 vini diversi, contribuendo per il 42.27% sul totale dei vini unici per le migliori 20 nazioni per media punti. Segue la francia con 19.739 (16.61%). <br>Lo stato al secondo posto ha meno della metà dei vini rispetto alla nazione al primo posto. <br>Sull'ultimo gradino del podio troviamo l'Italia con 17.805 (14.98%)
<br><br>US, Francia e Italia compongono oltre l'75% del dataset. <br>
Considerando il punteggio massimo dei vini, la media e il conteggio ho deciso che mi concentrerò su Francia, Italia, Us, Portogallo e Austria per l'assortimento eliminando il resto delle nazioni


In [ ]:
# Countries to keep
top_5 = ['France', 'Italy', 'US', 'Portugal', 'Austria']

# Filter
cond = df['country'].isin(top_5)

# Dataset
countries_df = df.loc[cond].copy()

# Visualization settings
countries_df.head().style.set_properties(subset=['description'], **{'width': '300px'})

### Distribuzione range di prezzo

In [ ]:
# Histogram
fig = px.histogram(
    countries_df, 
    x='price_range',
    y='price_range',
    facet_col='country',
    facet_col_wrap=2,
    histfunc='count',
    template='simple_white',
    text_auto=True,
    width=900,
    height=900
)

# Layout
fig.update_layout(
    xaxis=dict(
        title='Price distribution',
        categoryorder='category ascending'
    ),
    yaxis=dict(
        title='Count'
    )
)

fig.show()

Da questo grafico possiamo vedere come si distribuiscono i vini delle 5 top nazioni per quanto riguarda i 10 range di prezzo creati precedentemente. Anche filtrando per nazione, possiamo vedere come le bottiglie del range 0-9 siano quasi assenti contro invece lo smisurato aumento nel range 10-19 che va via via scendendo per poi avere un leggero incremento superata la soglia di 79.<br>L'unica differenza la fa l'US che ha come maggior concentrazione il range di prezzo 20-29

### Prezzo per punteggio

In [ ]:
countries_df.corr(numeric_only=True)

C'è una debole correlazione tra punteggio e prezzi, mentre non c'è nessuna correlazione tra anni e prezzo. Non c'è nessuna correlazione nemmeno tra anni e punti.<br> Vediamo come si comportano punti e prezzi tra loro per ogni nazione

In [ ]:
# Figure
fig = px.scatter(countries_df, x='points', y='price', facet_col='country', facet_col_wrap=3, width=900, height=900, opacity=0.2, trendline='ols', trendline_color_override='red')
fig.show()

# Trendline
results = px.get_trendline_results(fig)

# Summary tables
for t in top_5:
    print(t)
    print(results.query(f"country == '{t}'").px_fit_results.iloc[0].summary(), end='\n\n\n')

Da questo grafico possiamo osservare come la Francia sia quella con il vino più costoso, e in generale con i vini più costosi in assoluto tra le 5 nazioni. Osserviamo inoltre che il vino più costoso, ma in generale quelli con i prezzi più alti, non sono quelli con il punteggio maggiore

Ogni trendline ha un punteggio Adj. R-squared pari o inferiore al 20%, ciò significa che il modello lineare spiega solamente il 20% o meno della varianza. Anche il punteggio di Durbin-Watson è inferiore a 2 in tutti e 5 i casi. Non è assolutamente abbastanza nel caso in cui volessimo utilizzare un modello lineare per prevedere i prezzi in base al punteggio.

In [ ]:
fig = px.scatter(countries_df, x='points', y='price', color='country', width=900, height=900, trendline='ols', trendline_scope='overall', opacity=0.35, trendline_color_override='red')
fig.show()

Anche nel caso generale abbiamo un Adj. R-squared al 16%. Solamente il punteggio del vino non è abbastanza per provare a prevedere il prezzo di un vino.

In [ ]:
# Year - Price scatter plot
fig = px.scatter(countries_df, x='year', y='price', facet_col='country', facet_col_wrap=3, width=900, height=900, opacity=0.35)
fig.show()

Questo grafico conferma l'assenza di correlazione tra prezzo e anno di produzione, nonostante il luogo comune suggerisca che più un vino è vecchio più costa.

Dato che non sono riuscito a prevedere i prezzi con un modello lineare, devo ancora occuparmi dei valori mancanti. Ho deciso quindi di riempire questi valori con la media prezzo dei vini con medesimo punteggio e nazione.

In [ ]:
countries_df['price'].fillna(countries_df.groupby(['country', 'points'])['price'].transform('mean'), inplace=True)

Controlliamo ora se è andato tutto a buon fine

In [ ]:
countries_df['price'].isna().sum()

Per qualche motivo abbiamo ancora un valore mancante, vediamo di cosa si tratta

In [ ]:
cond = countries_df['price'].isna()
countries_df.loc[cond]

Cerchiamo altri vini austriaci con punteggio pari a 98

In [ ]:
# Filter
cond = (countries_df['country'] == 'Austria') & (countries_df['points'] == 98)

# Filtering dataset
countries_df.loc[cond, 'price'].mean()

Non ci sono altri vini austriaci con un punteggio pari a 98, non è stato quindi possibile trovare un valore medio da associare al vino con queste caratteristiche<br>
Sostituirò il prezzo con la media dei valori medi per i vini con punteggio pari a 98 delle altre nazioni

In [ ]:
# Getting mean value
fill_value = countries_df[countries_df['points'] == 98].groupby('country')['price'].mean().mean()

# Value print
fill_value

In [ ]:
# Filling missin values
countries_df['price'].fillna(value=fill_value, inplace=True)

Costruisco una nuova colonna con il rapporto qualità/prezzo

In [ ]:
# Points/price column
countries_df['points/price'] = countries_df['points'] / countries_df['price']

Aggiorno la colonna dei range di prezzo con i 4 range di prezzo che costituiranno il mio catalogo:
<ul>
<li>0-17 - Basic</li>
<li>18-40 - Standard</li>
<li>41-79 - Premium</li>
<li>80+ - Elite</li>
</ul>

Ho scelto questi range perché controllando la distribuzione dei prezzi sotto 79 dollari (upper fence della distribuzione di tutti i prezzi) il 25% dei prezzi si colloca sotto 18, il 75% sotto 40, 79 è appunto l'upper fence. 

In [ ]:
# Updating price_range column
countries_df['price_range'] = countries_df['price'].apply(lambda x: 'Elite' if x > 79 else 'Premium' if x > 40 else 'Standard' if x > 17 else 'Basic')

# Dataset
countries_df

Rendiamo la colonna **price range** una colonna categorica. In questo modo quando andremo ad ordinare in base a questa colonna, l'ordine seguirà l'ordine della lista da noi fornita e non quello alfabetico

In [ ]:
# Column dtype
countries_df['price_range'] = pd.Categorical(countries_df['price_range'], ['Basic', 'Standard', 'Premium', 'Elite'])

Ora è giunto il momento di cotruire il catalogo:<br>
Inserirò 3 vini per fascia di prezzo per ogni nazione basandomi sui punti e sul rapporto qualità/prezzo. 

In [ ]:
# Dataframe index reset and setting the indexes variable
countries_df.reset_index(drop=True, inplace=True)
catalogue_indexes = []

# Getting list of indexes for every country and every price range
for country in top_5:
    for range in countries_df['price_range'].unique():
        cond = (countries_df['country'] == country) & (countries_df['price_range'] == range)
        indexes = countries_df.loc[cond].sort_values(by=['points', 'points/price'], ascending=False).head(3).index.to_list()
        catalogue_indexes += indexes

In [ ]:
# Building the catalog dataframe
catalogue_df = countries_df.iloc[catalogue_indexes].sort_values(by=['country', 'price_range', 'points/price'], ascending=[True, True, False])
catalogue_df.reset_index(drop=True, inplace=True)

L'ultimo passaggio è quello di arrotondare le colonne dei prezzi e dei points/price a due cifre decimali. Ecco il catalogo che comprende le 5 migliori nazioni, con 3 vini per categoria per un totale di 60 vini nel catalogo.

In [ ]:
catalogue_df = catalogue_df.round({'price':2, 'points/price':2})

catalogue_df

## Catalogo Variety

Ora passerò alla costruzione del catalogo per le migliori varietà e cantine.<br>
 I passaggi eseguiti sono i medesimi usati per la costruzione del catalogo per le migliori nazioni.

In [ ]:
# Aggregate functions (max, min, mean, count) on the dataframe grouped by variety on the points column sorted by mean column first

df.groupby('variety')['points'].agg(['max', 'min', 'mean', 'count']).sort_values(by=['mean', 'max', 'count'], ascending=[False, False, False])[:20]

Le migliori varietà per punteggio medio sono state recensite solo 1 o 2 volte nella maggior parte dei casi, con un punteggio massimo che non va oltre i 96 punti. Proviamo a cambiare i criteri di ordinamento dando la priorità al punteggio massimo ottenuto.

In [ ]:
# Aggregate functions (max, min, mean, count) on the dataframe grouped by variety on the points column sorted by max column first
df.groupby('variety')['points'].agg(['max', 'min', 'mean', 'count']).sort_values(by=['max', 'mean', 'count'], ascending=[False, False, False])[:20]

Ecco che anche questa volta la classifica prende una piega diversa, con delle varietà che sono state recensite centinaia di volte. Il range di punti ricevuti molto più ampio rispetto alla classifica precedente, oltre che alla grande quantità di recensioni ricevute è probabilmente dovuto anche alle differenti cantine che hanno prodotto tale varietà.

In [ ]:
# Getting top varieties list
top_varieties = df.groupby('variety')['points'].agg(['max', 'min', 'mean', 'count']).sort_values(by=['max', 'mean', 'count'], ascending=[False, False, False]).query('max == 100').index.to_list()

# Printing top varieties list
top_varieties

In [ ]:
# Top varieties dataframe
varieties_df = df.query('variety in (@top_varieties)').copy()

# Creating the plot figure
fig = px.histogram(varieties_df, x='points', y='points', histfunc='count', facet_col='variety', facet_col_wrap=5, height=900, width=1800)

# Printing the plot
fig.show()

Dagli istogrammi di distribuzione del conteggio dei punti, divisi per le top varietà vediamo come lo Chardonnay sia il padrone incontrastato seguito dal cabernet Sauvignon

Procedo a creare il catalogo che comprenderà 3 opzioni per fascia di prezzo per ogni varietà (13) per un totale di 156 vini

In [ ]:
# Filling missing values with the mena value of wines in the same variety and points
varieties_df['price'].fillna(varieties_df.groupby(['variety', 'points'])['price'].transform('mean'), inplace=True)

# Creating price range columns
varieties_df['price_range'] = varieties_df['price'].apply(lambda x: 'Elite' if x > 79 else 'Premium' if x > 40 else 'Standard' if x > 17 else 'Basic')

# Creating points/price column
varieties_df['points/price'] = varieties_df['points'] / varieties_df['price']

# Setting price_range column to categorical type
varieties_df['price_range'] = pd.Categorical(varieties_df['price_range'], ['Basic', 'Standard', 'Premium', 'Elite'])

# Resetting the index
varieties_df.reset_index(drop=True, inplace=True)

# Creating indexes list variable
variety_catalog_indexes = []

# Getting indexes and filling the list created above
for variety in top_varieties:
    for range in varieties_df['price_range'].unique():
        variety_indexes = varieties_df.query('variety == @variety and price_range == @range').sort_values(by=['points', 'points/price'], ascending=False).head(3).index.to_list()
        variety_catalog_indexes += variety_indexes

# Creating the dataframe of varieties, sorting by varities, price_range and points/price
variety_catalog_df = varieties_df.iloc[variety_catalog_indexes].sort_values(by=['variety', 'price_range', 'points/price'], ascending=[True, True, False])

# Resetting the index
variety_catalog_df.reset_index(drop=True, inplace=True)

# Rounding Price and points/price column 
variety_catalog_df = variety_catalog_df.round({'price':2, 'points/price':2})

# Priting the catalogue
variety_catalog_df